# Geospatial Image Classification with Deep Learning  
## End‑to‑End Satellite Image Analysis using CNNs and Vision Transformers

## Table of Contents

1. [The Dataset](#The-Dataset)

2. [Importing The Required Libraries and Data](#Importing-The-Required-Libraries-and-Data)
    - [TensorFlow Environment Settings](#TensorFlow-Environment-Settings)
    - [The Required Libraries](#The-Required-Libraries)
    - [Set Random Seed for Reproducibility](#Set-Random-Seed-for-Reproducibility)
    - [Check GPU Availability](#Check-GPU-Availability)
    - [Define Data Folder Path](#Define-Data-Folder-Path)

3. [Model Hyperparameters](#Model-Hyperparameters)

4. [Create Image Data Generator for Data Augmentation](#Create-Image-Data-Generator-for-Data-Augmentation)

5. [Create Training and Validation Generators](#Create-Training-and-Validation-Generators)

6. [Model Definition](#Model-Definition)

7. [Model Compilation](#Model-Compilation)

8. [Model Training](#Model-Training)

9. []

## The Dataset

## Importing The Required Libraries and Data

### TensorFlow Environment Settings

> Environment Variables:

- `TF_ENABLE_ONEDNN_OPTS` \
Controls Intel oneDNN CPU optimizations in TensorFlow.
  - `1` → enable optimized CPU kernels (default, faster)
  - `0` → disable them (useful for reproducibility or avoiding numerical differences)

- `TF_CPP_MIN_LOG_LEVEL` \
Controls how much TensorFlow logs to the console.
  - `0` → show all logs  
  - `1` → hide INFO  
  - `2` → hide INFO + WARNING  
  - `3` → show only errors  


Environment variables must be set before TensorFlow loads, otherwise they have no effect. This ensures TensorFlow reads those settings during initialization.

In [ ]:
import os

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

### The Required Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
import time
import shutil
import random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Flatten, Dropout,
    BatchNormalization, GlobalAveragePooling2D
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import HeUniform
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.metrics import accuracy_score

### Set Random Seed for Reproducibility  

In [ ]:
SEED = 62
random.seed(SEED) 
np.random.seed(SEED) 
tf.random.set_seed(SEED)

### Check GPU Availability

In [ ]:
device = "gpu" if tf.config.list_physical_devices('GPU') else "cpu"
print("Device available for training:", device)

### Define Data Folder Path

In [ ]:
data_path = os.path.join(".", "data") 
print("Data folder path:", data_path)

## Model Hyperparameters

Use a batch size that evenly divides the number of validation samples to prevent partial batches.



In [ ]:
# Model hyperparameters
img_w, img_h = 64, 64
n_channels = 3

batch_size = 120
lr = 1e-3           # Learning rate
n_epochs = 20       # Adjust as needed

model_name = "tf_model"

## Configure Image Data Generator for Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
    # Convert pixel values from the range [0,255] to [0,1]
    rescale= 1./255,
    # Randomly rotate images by up to ±25 degrees 
    rotation_range= 25, 
    # Randomly shift the image horizontally by up to 15% of the width
    width_shift_range= 0.2, 
    height_shift_range= 0.2, 
    # Apply a shearing transformation, like slanting the image
    shear_range= 0.2,
    # Randomly zoom in or out by up to 20%
    zoom_range= 0.2, 
    # Randomly flip images left–right
    horizontal_flip= True, 
    vertical_flip= True,
    # Determine how to fill in new pixels created by rotations, shifts, or zooms
    # "nearest" copies the value of the nearest pixel
    fill_mode= "nearest",
    validation_split= 0.2,
)

## Create Training and Validation Generators

In [ ]:
train_generator = datagen.flow_from_directory(
    data_path,
    target_size= (img_w, img_h),
    batch_size= batch_size,
    class_mode= "binary",  # "categorical" for multi-class
    subset= "training",
    shuffle= True,
)

val_generator = datagen.flow_from_directory(
    data_path,
    target_size= (img_w, img_h),
    batch_size= batch_size,
    class_mode= "binary",  # "categorical" for multi-class
    subset= "validation",
    shuffle= False,
)


> Why ``shuffle=True`` for training but ``shuffle=False`` for validation

Shuffling is enabled for the training generator because the model should see 
the data in a different order in each epoch to improve generalization and to reduce overfitting, while the validation generator keeps ``shuffle=False`` so evaluation remains stable and deterministic. Even with a fixed seed, shuffling the validation set is still discouraged because the goal of validation is stable, repeatable evaluation. A seed only guarantees that the shuffle order is the same each run, but the order would still change every epoch.

For ``flow_from_directory``, the default value of ``shuffle`` is ``True``.

## Model Definition

In [ ]:
def build_cnn(input_shape, num_classes= 1):
    model = Sequential([
        # --- Convolution Block 1 ---
        Conv2D(32, 
               (5, 5), 
               activation= "relu", 
               padding= "same",
               strides= (1, 1), 
               kernel_initializer= HeUniform(),
               input_shape= input_shape),
        MaxPooling2D((2, 2)),
        BatchNormalization(),

        # --- Convolution Block 2 ---
        Conv2D(64, 
               (5, 5), 
               activation="relu", 
               padding= "same",
               strides= (1, 1), 
               kernel_initializer= HeUniform()),
        MaxPooling2D((2, 2)),
        BatchNormalization(),

        # --- Convolution Block 3 ---
        Conv2D(128, 
               (5, 5), 
               activation= "relu", 
               padding= "same",
               strides= (1, 1), 
               kernel_initializer= HeUniform()),
        MaxPooling2D((2, 2)),
        BatchNormalization(),

        # --- Convolution Block 4 ---
        Conv2D(256, 
               (5, 5), 
               activation= "relu", 
               padding= "same",
               strides= (1, 1), 
               kernel_initializer= HeUniform()),
        MaxPooling2D((2, 2)),
        BatchNormalization(),

        # --- Convolution Block 5 ---
        Conv2D(512, 
               (5, 5), 
               activation= "relu", 
               padding= "same",
               strides= (1, 1), 
               kernel_initializer= HeUniform()),
        MaxPooling2D((2, 2)),
        BatchNormalization(),

        # --- Convolution Block 6 ---
        Conv2D(1024, 
               (5, 5), 
               activation= "relu", 
               padding= "same",
               strides= (1, 1), 
               kernel_initializer= HeUniform()),
        MaxPooling2D((2, 2)),
        BatchNormalization(),

        # --- Global Pooling ---
        GlobalAveragePooling2D(),

        # --- Dense Block 1 ---
        Dense(64, 
              activation= "relu", 
              kernel_initializer= HeUniform()),
        BatchNormalization(),
        Dropout(0.4),

        Dense(128, 
              activation= "relu", 
              kernel_initializer= HeUniform()),
        BatchNormalization(),
        Dropout(0.4),

        Dense(256, 
              activation= "relu", 
              kernel_initializer= HeUniform()),
        BatchNormalization(),
        Dropout(0.4),

        # --- Dense Block 2 ---
        Dense(512, 
              activation= "relu", 
              kernel_initializer= HeUniform()),
        BatchNormalization(),
        Dropout(0.4),

        Dense(1024, 
              activation= "relu", 
              kernel_initializer= HeUniform()),
        BatchNormalization(),
        Dropout(0.4),

        Dense(2048, 
              activation= "relu", 
              kernel_initializer=HeUniform()),
        BatchNormalization(),
        Dropout(0.4),

        # --- Output Layer ---
        Dense(num_classes, activation="sigmoid")
    ])

    return model

## Model Compilation

In [ ]:
# Build the model
model = build_cnn(
    input_shape= (img_w, img_h, n_channels),
    num_classes= 1
)


In [ ]:
# Define loss function
loss_fn = "binary_crossentropy"

# Compile the model
model.compile(
    optimizer= Adam(learning_rate= lr),
    loss= loss_fn,
    metrics= ["accuracy"]
)

# Display model summary
model.summary()

In [ ]:
total_layers = len(model.layers)

print("Total layers:", len(model.layers))

>**Layers with no parameters** 

- MaxPooling

- GlobalAveragePooling

- Dropout

- Activation layers

## Training Setup

In [ ]:
# Steps per epoch 
steps_per_epoch = train_generator.samples // batch_size
validation_steps = val_generator.samples // batch_size

In [ ]:
# Callbacks
early_stop = EarlyStopping(
    monitor= "val_loss",
    patience= 8,
    restore_best_weights= True
)

reduce_lr = ReduceLROnPlateau(
    monitor= "val_loss",
    factor= 0.2,
    patience= 5,
    min_lr= 1e-6,
)

checkpoint = ModelCheckpoint(
    filepath= model_name + ".keras",
    monitor= "val_loss",
    save_best_only= True,
)

callbacks = [early_stop, reduce_lr, checkpoint]

Where EarlyStopping decides when to stop, ReduceLROnPlateau decides how fast the model should keep learning.

> What ``EarlyStopping`` does

- It monitors ``val_loss`` every epoch. If ``val_loss`` does not improve for patience consecutive epochs (``patience=8``), training stops early. It restores the best weights seen during training (``restore_best_weights=True``), not the weights from the final epoch.

<br><br>

> What ``ReduceLROnPlateau`` does

- ``ReduceLROnPlateau`` is the adaptive learning‑rate  controller. It watches a metric — in our case "``val_loss``" — and if that metric stops improving for a certain number of epochs (``patience=5``), it reduces the learning rate by a factor (``factor=0.2``). \
So with our settings:
  - If validation loss does not improve for 4 epochs then the learning rate becomes
  $$
  \text{new\_lr} = 0.2 \cdot \text{old\_lr}
  $$
  It keeps doing this until it reaches the minimum allowed learning rate (``min_lr=1e-6``).

<br><br>

> What ``ModelCheckpoint`` does

ModelCheckpoint is our automatic model saver.

It monitors the metric ``val_loss``. Every time ``val_loss`` reaches a new minimum, it saves the model to disk. It overwrites the previous checkpoint so we always keep the best model, not the last one.


## Model Training

In [ ]:
# ====================== Training Overview ======================

print(
    f"\n"
    f"================ Training Hyperparameters ================\n"
    f"  Device:                   {device}\n"
    f"  n_classes (train):        {train_generator.num_classes}\n"
    f"  n_classes (validation):   {val_generator.num_classes}\n"
    f"  Image Size:               ({img_w}, {img_h})\n"
    f"  n_channels:               {n_channels}\n"
    f"  batch_size:               {batch_size}\n"
    f"  steps_per_epoch:          {steps_per_epoch}\n"
    f"  validation_steps:         {validation_steps}\n"
    f"  n_epochs:                 {n_epochs}\n"
    f"  learning_rate:            {lr}\n"
    f"=========================================================="
)


In [ ]:
# ==== Model Training ====

history = model.fit(
    train_generator,
    epochs= n_epochs,
    steps_per_epoch= steps_per_epoch,
    validation_data= val_generator,
    validation_steps= validation_steps,
    callbacks= [early_stop, reduce_lr, checkpoint],
    verbose= 1
)

In [ ]:
model = load_model("tf_model.keras")
val_loss, val_acc = model.evaluate(val_generator)

print("Validation accuracy:", val_acc)
print("Validation loss:", val_loss)